## Running Example

### Prerequisite

Please install the libraries mentioned in the file requirements.txt
You can do it by using pip install -r requirements.txt 

In [ ]:
import pm4py
import processmining

import discover_occnets.py as occn_discover
import view_occnets.py as occn_vis

In [ ]:
file_path = './runningExample.jsonocel'

First, let us see how the process model looks like when including all object types.

In [ ]:
ot_activities, ot_seq, subgraphs = subgraphs_dict(file_path)
OCCN_model = all_ot_visualization(ot_activities, ot_seq, subgraphs)

To visualize the model, we can define a profile. If not defined the model will include all object types in the OCEL file. The profile is a list of the object types that we want to include in the model visualization. To do so, let us first check which object types are present in the business process contained in the log.

In [ ]:
print(ot_activities)

Now, we can select which object types should be included in the model. Let us say that we want to have customers, orders, items, and packages in the model.

In [ ]:
profile = ['customers', 'orders', 'items', 'packages']

In [ ]:
OCCN_model = all_ot_visualization(ot_activities, ot_seq, subgraphs, profile=profile)

In the model, some edges have the frequency of one activity followed by another in different colors. Each object type is assigned a color. When the dependency between two activities is found in different object types, it is represented by their different colors, for clarity.

When one activity follows the other for different object types, it is necessary to merge the edges without losing the frequency information of the dependency. This is done only for single bindings, not for compound bindings.

After filtering activities and edges that meet this requirement, the edges are joined.

Then, the corresponding labels are merged, keeping the original color of the object type it refers to.

Next, the nodes of merged edges are removed from the graph.

The final step is to generate the graph and create the legend.